In [8]:
# Se estiver rodando em um Colab, descomente a linha abaixo
# !pip install snscrape tqdm

In [9]:
from dataclasses import fields
import pandas as pd
import snscrape.modules.twitter as sntwitter
from tqdm import tqdm

# Oculta warnings do notebook
# import warnings
# warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 80)
pd.options.display.float_format = '{:,.2f}'.format

In [10]:
# Começamos filtrando todos os tweets de um usuario, dentro do período desejado
username = 'ErikakHilton'  # example user
since = '2022-11-01'
until = '2022-11-02'

query = f'from:{username} since:{since} until:{until}'
user_scrapping_results = sntwitter.TwitterSearchScraper(query)

tweet_ids = []
for tweet in tqdm(user_scrapping_results.get_items()):
    tweet_ids.append(tweet.id)

print(len(tweet_ids))

88it [00:05, 16.10it/s]

88


In [11]:
# Vamos iterar entre os id's para pegar os dados dos tweets e suas respostas
# Esta etapa pode demorar um pouco
tweets_and_replies = []

for tweet_id in tqdm(tweet_ids):
    tweet_scrapper = sntwitter.TwitterTweetScraper(tweet_id, mode=sntwitter.TwitterTweetScraperMode.SCROLL)
    for tweet in tweet_scrapper.get_items():
        tweets_and_replies.append(tweet)
        
print(len(tweets_and_replies))

100%|█████████████████████████████████████████████████████████████████████| 88/88 [01:53<00:00,  1.29s/it]

1513


In [34]:
# Convertendo as informações para DataFrame
tweet_fields = [f.name for f in fields(tweet)]
user_fields = [f.name for f in fields(tweet.user)]
all_columns = tweet_fields + ['user_' + f for f in user_fields]

tweets_data = [
    [getattr(t, field) for field in tweet_fields] + 
    [getattr(t.user, field) for field in user_fields] 
    for t in tweets_and_replies
]
tweets_df = pd.DataFrame(tweets_data, columns=all_columns)

print(tweets_df.shape)
tweets_df.head()

(1513, 49)


,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe,user_username,user_id,user_displayname,user_rawDescription,user_renderedDescription,user_descriptionLinks,user_verified,user_created,user_followersCount,user_friendsCount,user_statusesCount,user_favouritesCount,user_listedCount,user_mediaCount,user_location,user_protected,user_link,user_profileImageUrl,user_profileBannerUrl,user_label
0,https://twitter.com/ErikakHilton/status/158756...,2022-11-01 21:49:34+00:00,Zambelli cassada em breve?!,Zambelli cassada em breve?!,1587562519568125953,https://twitter.com/ErikakHilton,30,49,826,5,1587562519568125953,pt,None,None,None,None,None,None,https://twitter.com/AndreiaSadi/status/1587549...,NaN,None,None,None,None,None,None,None,None,None,ErikakHilton,738143559920934912,ERIKA HILTON 🚩 🇧🇷,Deputada Federal - PSOL/SP (2023-2026)\nVeread...,Deputada Federal - PSOL/SP (2023-2026)\nVeread...,None,True,2016-06-01 23:01:53+00:00,254875,4791,26139,64635,459,3181,"São Paulo, Brasil",False,"TextLink(text='erikahilton.com.br', url='http:...",https://pbs.twimg.com/profile_images/155938099...,https://pbs.twimg.com/profile_banners/73814355...,None
1,https://twitter.com/mattteusalves/status/15875...,2022-11-01 21:50:44+00:00,@ErikakHilton Erika faça o que for preciso par...,@ErikakHilton Erika faça o que for preciso par...,1587562813236576258,https://twitter.com/mattteusalves,1,0,4,0,1587562519568125953,pt,None,None,None,None,None,None,None,"1,587,562,519,568,125,952.00",https://twitter.com/ErikakHilton,[https://twitter.com/ErikakHilton],None,None,None,None,None,None,None,mattteusalves,1247019452253843457,matheus,Sempre em dúvida,Sempre em dúvida,None,False,2020-04-06 04:34:29+00:00,120,797,25514,134718,1,11,"Recife, Brasil",False,None,https://pbs.twimg.com/profile_images/161020706...,None,None
2,https://twitter.com/agenteputativa/status/1587...,2022-11-01 21:51:01+00:00,@ErikakHilton eu li 'casada' \ne pensei 'ué ou...,@ErikakHilton eu li 'casada' \ne pensei 'ué ou...,1587562883264794632,https://twitter.com/agenteputativa,0,0,1,0,1587562519568125953,pt,None,None,None,None,None,None,None,"1,587,562,519,568,125,952.00",https://twitter.com/ErikakHilton,[https://twitter.com/ErikakHilton],None,None,None,None,None,None,None,agenteputativa,754146561488252928,Polly,O próprio caos,O próprio caos,None,False,2016-07-16 02:52:06+00:00,102,798,2853,13000,0,164,"Salvador, Brasil",False,None,https://pbs.twimg.com/profile_images/158630175...,https://pbs.twimg.com/profile_banners/75414656...,None
3,https://twitter.com/HughOnMusic/status/1587563...,2022-11-01 21:51:29+00:00,@ErikakHilton Meu sonho,@ErikakHilton Meu sonho,1587563001221185537,https://twitter.com/HughOnMusic,0,0,0,0,1587562519568125953,pt,None,None,None,None,None,None,None,"1,587,562,519,568,125,952.00",https://twitter.com/ErikakHilton,[https://twitter.com/ErikakHilton],None,None,None,None,None,None,None,HughOnMusic,29006610,it'svitorhugo_carvalho ✌🏻🍂🍁,"Farmacêutico, geminiano, gamer e músico","Farmacêutico, geminiano, gamer e músico",None,False,2009-04-05 15:48:48+00:00,263,730,21909,13091,3,1351,"São Paulo, Brasil",False,"TextLink(text='i.instagram.com/hughonmusic/', ...",https://pbs.twimg.com/profile_images/100050745...,https://pbs.twimg.com/profile_banners/29006610...,None
4,https://twitter.com/j000rdy/status/15875630344...,2022-11-01 21:51:37+00:00,@ErikakHilton DRA ERIKA HILTON POR FAVOR CUIDE...,@ErikakHilton DRA ERIKA HILTON POR FAVOR CUIDE...,1587563034498674688,https://twitter.com/j000rdy,0,0,1,0,1587562519568125953,es,None,None,None,None,None,None,None,"1,587,562,519,568,125,952.00",https://twitter.com/ErikakHilton,[https://twitter.com/ErikakHilton],None,None,None,None,None,None,None,j000rdy,1341892997965897728,Jordy,,,None,False,2020-12-23 23:46:48+00:00,20,461,3044,72

In [33]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513 entries, 0 to 1512
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   url                       1513 non-null   object             
 1   date                      1513 non-null   datetime64[ns, UTC]
 2   rawContent                1513 non-null   object             
 3   renderedContent           1513 non-null   object             
 4   id                        1513 non-null   int64              
 5   user                      1513 non-null   object             
 6   replyCount                1513 non-null   int64              
 7   retweetCount              1513 non-null   int64              
 8   likeCount                 1513 non-null   int64              
 9   quoteCount                1513 non-null   int64              
 10  conversationId            1513 non-null   int64              
 11  lang             

In [35]:
tweets_df.iloc[0]

url                         https://twitter.com/ErikakHilton/status/158756...
date                                                2022-11-01 21:49:34+00:00
rawContent                                        Zambelli cassada em breve?!
renderedContent                                   Zambelli cassada em breve?!
id                                                        1587562519568125953
user                                         https://twitter.com/ErikakHilton
replyCount                                                                 30
retweetCount                                                               49
likeCount                                                                 826
quoteCount                                                                  5
conversationId                                            1587562519568125953
lang                                                                       pt
source                                                          